# 概要
* [ JVS ](https://sites.google.com/site/shinnosuketakamichi/research-topics/jvs_corpus)のデータで評価を行う
* Amazon Transcribe で文字起こしした結果を Word Error Rate で評価する
* まずはデータの準備を行う
* 定期的にTranscribeの評価を行う場合はこのノートブックは初回のみ実行するだけでよい

# このノートブックの成果物
* 文字起こしをしたいwavファイル（がS3に置かれる）
* wavファイル名と正解の文字起こしデータのcsv（がローカルにおかれる）

In [1]:
# 定数

# 音声ファイルを配置するバケット名
BUCKET_NAME = 'transcribe-input-bucket-202004161130'

In [ ]:
# データのダウンロード
!chmod 755 ./download.sh
!./download.sh

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   408    0   408    0     0   1159      0 --:--:-- --:--:-- --:--:--  1159
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:01 --:--:--     0

In [ ]:
# ダウンロードしたデータの解凍
!unzip audio.zip | head -n10

In [ ]:
# 解凍したファイルとディレクトリ構成のチェック
!ls -l jvs_ver1/jvs001/falset10

In [ ]:
!cat jvs_ver1/jvs001/falset10/transcripts_utf8.txt

In [ ]:
!ls -l jvs_ver1/jvs001/falset10/wav24kHz16bit

# 中身を見てわかったこと
* 文字起こしした結果はあっても音声ファイルのみが存在する場合がある(BASIC5000_1485はラベルだけあって音声がない)
* 音声ファイルのみあって文字起こし結果がない場合もある(BASIC5000_1635は音声のみでラベルがない）
* 音声ファイルの上のディレクトリにあるtranscripts_utf8.txtに、ファイル名（拡張子なし）と文字起こし結果がコロン区切りで格納されている

# ファイルパスの前処理
* Transcribeの特性上、job名.jsonでoutputバケット直下に結果が出力される
* job名にスラッシュが使えないため、audioファイルをすべてディレクトリ直下に配置し、inputバケット直下にコピーし、ファイル名をジョブ名とする
* 併せて、ラベルデータとファイル名のペアをcsvで作成する

In [ ]:
# 指定ディレクトリ以下を深さ優先探索する関数
import os
def find_all_files(directory):
    for root, dirs, files in os.walk(directory):
        yield root
        for file in files:
            yield os.path.join(root, file)

In [ ]:
# 音声ファイルと文字起こしテキストのリストを取得
wav_file_path_list = []
transcribe_utf8_list = []
for file in find_all_files('./jvs_ver1/'):
    if '.wav' in file:
        wav_file_path_list.append(file)
    elif 'transcripts_utf8.txt' in file:
        transcribe_utf8_list.append(file)
wav_file_path_list = sorted(wav_file_path_list)
transcribe_utf8_list = sorted(transcribe_utf8_list)
print(wav_file_path_list[:10])
print(transcribe_utf8_list[:10])

In [ ]:
# 音声ファイルリストから move 先のファイル名を生成する
wav_file_name_list = []
for wav_file in wav_file_path_list:
    file_name = wav_file.split('/')
    file_name = file_name[2] + '--' + file_name[3] + '--' + file_name[5]
    wav_file_name_list.append(file_name)
wav_file_name_list = sorted(wav_file_name_list)
print(wav_file_name_list[:10])

# transcripts_utf8.txtの整形
* wav_file_name_list に合わせた形で文字起こしデータを作成する

In [ ]:
transcript_data_list = []
transcript_file_name_list = []
for path in transcribe_utf8_list:
    with open(path) as f:
        fix_path = path.split('/')
        fix_path = fix_path[2] + '--' + fix_path[3] + '--'
        text = f.read()
        lines = text.split('\n')
        for line in lines:
            if line == '':
                pass
            else:
                d = line.split(':')
                transcript_file_name_list.append(fix_path+d[0]+'.wav')
                transcript_data_list.append(d[1])
print(transcript_file_name_list[:10])
print(transcript_data_list[:10])

# 音声ファイルとtranscriptファイルが合致するものだけ残す

In [ ]:
import pandas as pd

In [ ]:
label_temp_df = pd.DataFrame({
    'file_name':transcript_file_name_list,
    'transcript_data':transcript_data_list
})
label_temp_df.head()

In [ ]:
wav_df = pd.DataFrame({
    'file_path':wav_file_path_list,
    'file_name':wav_file_name_list
})
wav_df.head()

In [ ]:
label_df = pd.merge(wav_df, label_temp_df)
label_df.head()
print(len(label_df))

# 音声ファイルとtranscript_dataが一致していないものを除外する
* 一部ずれているファイルがあるので除外
* NGリストを別途準備した

In [ ]:
with open('./NG_list.txt') as f:
    NG_list = f.readlines()
    for NG_txt in NG_list:
        NG_txt = NG_txt.replace('\n','')
        label_df.drop(label_df[label_df['file_name'] == NG_txt].index, inplace=True)
label_df = label_df.reset_index()[['file_path','file_name','transcript_data']]

In [ ]:
len(label_temp_df),len(label_df)

In [ ]:
# 音声ファイルをリネーム（コピーだけど）して保存するフォルダを用意
!mkdir -p wav

In [ ]:
import shutil

# ファイルを移動
for i,r in label_df.iterrows():
    src = r[0]
    dest = './wav/' + r[1]
    shutil.copyfile(src,dest)

In [ ]:
label_df.to_csv('./label.csv',index=False)

In [ ]:
pd.read_csv('./label.csv').head()

In [ ]:
# S3にアップロード
import boto3
resource = boto3.resource('s3')
bucket = resource.Bucket(BUCKET_NAME)

In [ ]:
for file in list(label_df['file_name'].values):
    src = './wav/' + file
    bucket.upload_file(src,file)

In [ ]:
len(pd.read_csv('./label.csv'))